In [79]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from models import NN, Dummy
from utils import add_params, scale_params, subtract_params
import torch

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
model1 = NN(1,1,1)
model2 = NN(1,1,1)
a = torch.tensor([1.])

print('----- model 1 -----')
for param in model1.parameters():
    print(param)

print('\n----- model 2 -----')
for param in model2.parameters():
    print(param)

print(f'\nOutput before weight adjustments : {model1(a)}')

print('\n----- model adjusted -----')
copy_of_params = model1.parameters()
scale_params(copy_of_params, a=2)
# new_params = add_params(new_params, model2.parameters())
for param in model1.parameters():
    print(param)

print(f'\nOutput after weight adjustments : {model1(a)}')

----- model 1 -----
Parameter containing:
tensor([[0.7851]], requires_grad=True)
Parameter containing:
tensor([-0.7002], requires_grad=True)
Parameter containing:
tensor([[0.1879]], requires_grad=True)
Parameter containing:
tensor([0.2036], requires_grad=True)

----- model 2 -----
Parameter containing:
tensor([[-0.4236]], requires_grad=True)
Parameter containing:
tensor([-0.8657], requires_grad=True)
Parameter containing:
tensor([[0.5011]], requires_grad=True)
Parameter containing:
tensor([0.5872], requires_grad=True)

Output before weight adjustments : tensor([0.2195], grad_fn=<AddBackward0>)

----- model adjusted -----
Parameter containing:
tensor([[1.5702]], requires_grad=True)
Parameter containing:
tensor([-1.4004], requires_grad=True)
Parameter containing:
tensor([[0.3759]], requires_grad=True)
Parameter containing:
tensor([0.4072], requires_grad=True)

Output after weight adjustments : tensor([0.4710], grad_fn=<AddBackward0>)


### ADMM Global Consensus

In [81]:
rho = 1

# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)
lambdas = np.vstack([lam, -lam])

agents = [
    GlobalConsensus(
        rho=rho, 
        x_init=np.random.randn(2), 
        lam_init=lam
    ) 
    for lam in lambdas
]

# comopute initial average
avg = 0
for agent in agents:
    avg += agent.x/len(agents) 

# broadcast average to all agents
for agent in agents:
    agent.primal_average = avg
    print(agent.x)
print(f'average = {agents[0].primal_average}')

[1.66627408 0.36283321]
[-0.62562115 -1.06675469]
[-0.87154073 -1.16677911]
[-0.85410717 -0.75969017]
average = [-0.17124875 -0.65759769]


In [82]:
for i in range(50):
    # compute primal variables
    for agent in agents:
        agent.primal_update()

    # compute new global average
    avg = 0
    for agent in agents:
        avg += agent.x/len(agents) 
    
    # broadcast average to all agents
    for agent in agents:
        agent.primal_average = avg

    # update dual variables    
    for agent in agents:
        agent.dual_update()

for i, agent in enumerate(agents):
    print(f'agent {i}: x = {agent.x}, lam = {agent.lam}')

agent 0: x = [-9.29896088e-10  2.54475796e-10], lam = [ 1.85979218e-09 -5.08951591e-10]
agent 1: x = [-9.95162857e-11 -2.62032839e-10], lam = [1.99032571e-10 5.24065679e-10]
agent 2: x = [ 9.29896088e-10 -2.54475796e-10], lam = [-1.85979218e-09  5.08951591e-10]
agent 3: x = [9.95162857e-11 2.62032839e-10], lam = [-1.99032571e-10 -5.24065679e-10]


### Event-Based ADMM Global Consensus

In [83]:
rho = 1

deltas = [0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
deltas = [1e-5]
t_max = 100

# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)*5
lambdas = np.vstack([lam, -lam])
x_init = np.random.randn(lambdas.shape[0],2)*5
initial_avg = np.mean(x_init, axis=0)

In [84]:
for delta in deltas:

    # Initialise Agents
    agents = [
        EventGlobalConsensus(
            N = len(lambdas), 
            rho=rho, 
            delta=delta,
            x_init=x, 
            lam_init=lam
        ) 
        for lam, x in zip(lambdas, x_init)
    ]

    # broadcast average to all agents
    for agent in agents:
        agent.primal_avg = initial_avg

    # Run event based ADMM
    comm = 0

    for t in range(t_max):
        
        for agent in agents:
            agent.primal_update()
        
        sum_of_res = 0
        for agent in agents:
            if agent.broadcast: 
                comm += 1
                sum_of_res += agent.residual
            
        # This is somehow updating all the agents
        agent.primal_avg += sum_of_res
        
        for agent in agents:
            agent.dual_update()
    
    accuracy = np.sum([np.linalg.norm(agent.x - agent.C, ord=1) for agent in agents])
    
    load = comm/(t_max*len(agents))
    print(f'Accuracy = {accuracy:.6f}, load = {load}')
    

Accuracy = 0.000000, load = 0.3


### Event-Based ADMM with Torch

In [85]:
delta
t_max = 25

# Initial lambdas must sum to 0!
lam = np.random.randn(2,2)*5
lambdas = np.vstack([lam, -lam])
x_init = np.random.randn(lambdas.shape[0],2)*5

In [86]:
agents = [
    EventGlobalConsensusTorch(
        N = len(lambdas), 
        rho=rho, 
        model=Dummy,
        loss=torch.norm,
        delta=delta,
        x_init=x, 
        lam_init=lam
    ) 
    for lam, x in zip(lambdas, x_init)
]





TypeError: Dummy.__init__() missing 1 required positional argument: 'x'